In [1]:
#json文件，遍历每个元素，去掉rewrite_query的键
import json

# 假设你的JSON文件名是 'data.json'
with open('exaggerated-safety_xstest_v2_prompts1_our_work.json', 'r', encoding='utf-8') as file:
    data = json.load(file)
for item in data:
    if 'selfDefine_benign_prompt' in item:
        del item
# 遍历每个元素并删除 'rewrite_query' 键
# data = [item for item in data if 'rewrite_query' not in item]

# 如果你想保存修改后的数据到新的文件中
with open('exaggerated-safety_xstest_v2_prompts1_our_work.json', 'w', encoding='utf-8') as file:
    json.dump(data, file, ensure_ascii=False, indent=4)


In [8]:
# our_work/ThirdVersion/test_data/qwen_plus/benign/file/test.ipynb
import os
import json

# 文件夹路径
folder_path = '/mnt/workspace/our_work/ThirdVersion/result/qwen/topk=5/threshold=0.3/gte_Qwen2-1.5B-instruct/benign'

# 遍历文件夹中的所有 JSON 文件
for filename in os.listdir(folder_path):
    if filename.endswith('.json'):
        file_path = os.path.join(folder_path, filename)
        
        try:
            # 读取 JSON 文件
            with open(file_path, 'r', encoding='utf-8') as file:
                data = json.load(file)
            
            # 检查数据是列表还是字典，并进行处理
            if isinstance(data, list):
                # 对于列表中的每个元素，删除指定字段
                for item in data:
                    if isinstance(item, dict):
                        for key in ['judge', 'is harmful', 'reasoning', 'second judge','second result']:
                        # for key in ['judge', 'is harmful', 'reasoning', 'second judge','second result','scores','similar_pattern','similar_prompt']:
                            if key in item:
                                del item[key]
            elif isinstance(data, dict):
                # 如果是字典，则直接删除指定字段
                for key in ['judge', 'is_harmful', 'reasoning', 'second_judge','result']:
                    if key in data:
                        del data[key]

            # 保存修改后的文件
            with open(file_path, 'w', encoding='utf-8') as file:
                json.dump(data, file, ensure_ascii=False, indent=4)
            
            print(f"Processed and updated: {filename}")

        except Exception as e:
            print(f"Error processing {filename}: {e}")


Processed and updated: Cipher_benign_100_prompt_our_work_test.json
Processed and updated: ReNeLLM_benign_100_prompt_data_our_work.json
Processed and updated: exaggerated-safety_xstest_v2_prompts.json
Processed and updated: flip_benign.json
Processed and updated: jailbroken_benign_100_prompt_data_our_work.json
Processed and updated: multiLang_benign_prompt_our_work.json


In [2]:
import json

# 定义标记和替换文本
start_marker = "Here are some examples"
end_marker = "Please reply me the following prompt:"
replacement_text = "(with a shift of three position)"

# 读取 JSON 文件
with open('Cipher_benign_100_prompt_our_work_test.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

# 遍历 JSON 中的每个元素（假设每个元素为字典）
for item in data:
    # 获取第一个 key
    first_key = next(iter(item))
    value = item[first_key]
    
    # 确保 value 为字符串且包含起始标记
    if isinstance(value, str) and start_marker in value:
        # 尝试查找结束标记的位置
        end_index = value.find(end_marker, value.find(start_marker))
        if end_index != -1:
            # 找到起始标记的位置
            start_index = value.find(start_marker)
            # 替换从 start_marker（包含）到 end_marker（不包含）之间的部分
            new_value = value[:start_index] + replacement_text + value[end_index:]
            item[first_key] = new_value
        else:
            # 如果没有找到结束标记，则可根据需求处理（此处选择不做替换）
            pass

# 将处理后的结果写入新文件
with open('Cipher_benign_100_prompt_our_work_test.json', 'w', encoding='utf-8') as f:
    json.dump(data, f, ensure_ascii=False, indent=2)


In [ ]:
# 遍历json文件的元素，对于第一个Key如果不是Caesar_benign_prompt，则删除该Key的value中的(with a shift of three position)值

In [5]:
#取前420，取前500
with open('jailbroken_benign_100_prompt_data_our_work.json', 'r', encoding='utf-8') as f:
    data1 = json.load(f)

with open('multiLang_benign_prompt_our_work.json', 'r', encoding='utf-8') as f:
    data2 = json.load(f)

data1=data1[0:480]
data2=data2[0:500]
with open('jailbroken_benign_100_prompt_data_our_work.json', 'w', encoding='utf-8') as f:
    json.dump(data1, f, ensure_ascii=False, indent=2)


with open('multiLang_benign_prompt_our_work.json', 'w', encoding='utf-8') as f:
    json.dump(data2, f, ensure_ascii=False, indent=2)


In [10]:
#对比方法良性拒绝率
#读取文件夹下的json文件，对于每个json文件，有三列值，"IA rule judge":true的个数，json文件的总元素，"IA rule judge":true的个数/总个数（良性拒绝率），最后将每个json文件的这三个值保存为csv文件

import os
import json
import pandas as pd

def calculate_benign_rejection_rate(folder_path, output_csv):
    results = []
    
    # 遍历文件夹下的所有 JSON 文件
    for filename in os.listdir(folder_path):
        if filename.endswith(".json"):
            file_path = os.path.join(folder_path, filename)
            
            # 读取 JSON 文件
            with open(file_path, 'r', encoding='utf-8') as file:
                data = json.load(file)
            
            # 计算 "IA rule judge": true 的个数
            ia_rule_true_count = sum(1 for item in data if item.get("IA rule judge") is True)
            total_count = len(data)
            
            # 计算良性拒绝率
            rejection_rate = ia_rule_true_count / total_count if total_count > 0 else 0
            
            # 记录结果
            results.append({
                "Filename": filename,
                "IA_rule_true_count": ia_rule_true_count,
                "Total_count": total_count,
                "Benign_rejection_rate": rejection_rate
            })
    
    # 转换为 DataFrame 并保存为 CSV
    df = pd.DataFrame(results)
    df.to_csv(output_csv, index=False, encoding='utf-8')
    
    print(f"Results saved to {output_csv}")

# 使用示例
folder_path = "/mnt/workspace/our_work/dataset-25-1-16/wildjailbreak_final/defence/benign_test/benign_result/Self-Reminder/json"  # 替换为实际的文件夹路径
output_csv = "/mnt/workspace/our_work/dataset-25-1-16/wildjailbreak_final/defence/benign_test/benign_result/Self-Reminder/json/benign_rejection_rate.csv"
calculate_benign_rejection_rate(folder_path, output_csv)


Results saved to /mnt/workspace/our_work/dataset-25-1-16/wildjailbreak_final/defence/benign_test/benign_result/Self-Reminder/json/benign_rejection_rate.csv
